In [1]:
# Package Setup
import Pkg;
# Pkg.update()

# Allow for package development
using Revise

# Julia Packages
using Distributions
using LinearAlgebra
using Statistics
using PGFPlots

# Temporary for now
# Pkg.add("JuMP")
# Pkg.add("Gurobi")
using JuMP
using Gurobi

# Satellite Dynamics Packages
using SatelliteDynamics

# Load SatelliteTasking - Reclone to keep version current
# Pkg.clone("..") # For some reason this doens't work with Pkg.add + PackageSpec. Why?
using SatelliteTasking
using SatelliteTasking.SatellitePlanning
using SatelliteTasking.Analysis

┌ Info: Recompiling stale cache file /Users/deddy/.julia/compiled/v1.1/Revise/M1Qoh.ji for Revise [295af30f-e4ad-537b-8983-00126c2a3abe]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /Users/deddy/.julia/compiled/v1.1/Distributions/xILW0.ji for Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /Users/deddy/.julia/compiled/v1.1/PGFPlots/XxINN.ji for PGFPlots [3b7a836e-365b-5785-a47d-02c71176b4aa]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /Users/deddy/.julia/compiled/v1.1/SatelliteDynamics/9LFbr.ji for SatelliteDynamics [0e7c1a32-1b9f-5532-88a4-e668712d6a4c]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /Users/deddy/.julia/compiled/v1.1/SatelliteTasking/HFrko.ji for SatelliteTasking [30f2e302-0ae1-11e9-1cda-0f4324764595]
└ @ Base loading.jl:1184


  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/Stanford/repos/SatelliteTasking.jl/Project.toml`
 [no changes]
  Updating `~/Stanford/repos/SatelliteTasking.jl/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/Stanford/repos/SatelliteTasking.jl/Project.toml`
 [no changes]
  Updating `~/Stanford/repos/SatelliteTasking.jl/Manifest.toml`
 [no changes]


In [2]:
# Load Image Data

# Load test images
# @time images = load_images("../data/landsat_test_150.json", collect_duration=5.0);
@time images = load_images("../data/landsat_test_300.json", collect_duration=5.0);
# @time images = load_images("../data/landsat_test_600.json", collect_duration=5.0);
num_images = length(images)

println("Loaded $num_images images.")

  0.399942 seconds (1.23 M allocations: 64.049 MiB, 6.39% gc time)
Loaded 150 images.


In [8]:
# Simulation Definition

# Configure simulation
epc0 = Epoch(2019, 1, 1, 0, 0, 0, tsys=:UTC) # Start of time span
epcf = Epoch(2019, 1, 2, 0, 0, 0, tsys=:UTC) # End of simulation time span

# Set Simulation Time Step
timestep = 1

# Define Satellite Orbit
oe   = [R_EARTH + 500e3, 0, 90.0, 0, 0, 0]
eci0 = sOSCtoCART(oe, use_degrees=true)

# Spacecraft model parameters
mass      = 100.0 # [kg]
area_drag = 1.0   # [m^2]
coef_drag = 2.3
area_srp  = 1.0   # [m^2]
coef_srp  = 1.8

# Define force model parameters
n_grav = 20
m_grav = 20
drag   = true
srp    = true
sun    = true
moon   = true
relativity = true

true

In [4]:
# Simulate Orbit

# Simulate True Orbit
@time orbit = Orbit(epc0, epcf, eci0, timestep=timestep,
                    mass=mass, area_drag=area_drag, coef_drag=coef_drag, 
                    area_srp=area_srp, coef_srp=coef_srp, 
                    n_grav=n_grav, m_grav=m_grav, 
                    drag=drag, srp=srp, moon=moon, sun=sun, relativity=relativity)

 68.577802 seconds (1.38 G allocations: 34.493 GiB, 8.09% gc time)


Orbit(0)

In [11]:
# Compute Opportunities & Collects

# Compute Opportunities
@time true_opportunities = find_all_opportunities(orbit, images, sort=true);

# Compute Split Opportunities
@time opportunities = split_opportunities(true_opportunities, 3);

 12.566471 seconds (168.49 M allocations: 17.781 GiB, 15.38% gc time)
  0.001881 seconds (11.90 k allocations: 670.422 KiB)


In [6]:
# Graph Planning

